<a href="https://colab.research.google.com/github/aneez-hassan/PRODIGY_GA_05/blob/main/stylized_output.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Neural Style Transfer with Automatic Example Images
# Run this in Google Colab

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, models
from PIL import Image
import matplotlib.pyplot as plt
import requests
from io import BytesIO

# Function to load image from URL and transform it
def load_image_from_url(url, max_size=400, shape=None):
    response = requests.get(url)
    image = Image.open(BytesIO(response.content)).convert('RGB')

    if max(image.size) > max_size:
        size = max_size
    else:
        size = max(image.size)

    if shape is not None:
        size = shape

    in_transform = transforms.Compose([
        transforms.Resize(size),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406),  # Imagenet mean
                             (0.229, 0.224, 0.225))  # Imagenet std
    ])

    # Apply transform and add batch dimension
    image = in_transform(image).unsqueeze(0)
    return image

# Function to convert tensor to image for display
def im_convert(tensor):
    image = tensor.to("cpu").clone().detach()
    image = image.squeeze(0)
    image = image * torch.tensor([0.229, 0.224, 0.225]).view(3,1,1) + torch.tensor([0.485, 0.456, 0.406]).view(3,1,1)
    image = image.clamp(0,1)
    image = image.permute(1, 2, 0).numpy()
    return image

# URLs for example content and style images
content_url = "https://pytorch.org/tutorials/_static/img/neural-style/picasso.jpg"
style_url = "https://pytorch.org/tutorials/_static/img/neural-style/dancing.jpg"

# Load images
content = load_image_from_url(content_url)
style = load_image_from_url(style_url, shape=[content.size(2), content.size(3)])

# Display content and style images
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 8))
ax1.imshow(im_convert(content))
ax1.set_title('Content Image')
ax1.axis('off')
ax2.imshow(im_convert(style))
ax2.set_title('Style Image')
ax2.axis('off')
plt.show()

# Load VGG19 model pretrained on ImageNet
vgg = models.vgg19(pretrained=True).features

# Freeze parameters since we only want to optimize the target image
for param in vgg.parameters():
    param.requires_grad_(False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vgg.to(device)

# Define layers to use for content and style
content_layers = ['21']  # relu4_2
style_layers = ['0', '5', '10', '19', '28']  # relu1_1, relu2_1, relu3_1, relu4_1, relu5_1

# Function to get features from layers
def get_features(image, model, layers=None):
    features = {}
    x = image.to(device)
    for name, layer in model._modules.items():
        x = layer(x)
        if name in layers:
            features[name] = x
    return features

# Function to calculate Gram Matrix for style representation
def gram_matrix(tensor):
    batch, d, h, w = tensor.size()
    tensor = tensor.view(d, h * w)
    gram = torch.mm(tensor, tensor.t())
    return gram

# Get content and style features
content_features = get_features(content, vgg, content_layers)
style_features = get_features(style, vgg, style_layers)

# Calculate style Gram matrices
style_grams = {layer: gram_matrix(style_features[layer]) for layer in style_features}

# Create a target image and initialize it with content image
target = content.clone().requires_grad_(True).to(device)

# Define weights for style layers and content
style_weights = {'0': 1.0,
                 '5': 0.75,
                 '10': 0.2,
                 '19': 0.2,
                 '28': 0.2}

content_weight = 1e4
style_weight = 1e2

# Define optimizer
optimizer = optim.Adam([target], lr=0.003)

# Run style transfer
steps = 300  # Increase for better quality

for step in range(steps):
    target_features = get_features(target, vgg, content_layers + style_layers)

    # Content loss
    content_loss = torch.mean((target_features[content_layers[0]] - content_features[content_layers[0]])**2)

    # Style loss
    style_loss = 0
    for layer in style_layers:
        target_feature = target_features[layer]
        target_gram = gram_matrix(target_feature)
        style_gram = style_grams[layer]
        layer_style_loss = style_weights[layer] * torch.mean((target_gram - style_gram)**2)
        style_loss += layer_style_loss / (target_feature.numel())

    # Total loss
    total_loss = content_weight * content_loss + style_weight * style_loss

    # Update target image
    optimizer.zero_grad()
    total_loss.backward()
    optimizer.step()

    # Print loss every 50 steps
    if step % 50 == 0:
        print(f"Step {step}, Total loss: {total_loss.item():.4f}")

# Display final stylized image
final_img = im_convert(target)
plt.figure(figsize=(10,10))
plt.imshow(final_img)
plt.title("Stylized Image")
plt.axis('off')
plt.show()
